<a href="https://colab.research.google.com/github/aniketSanyal/DifferentialPrivacy/blob/main/Prefix_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

class Config:
  num_train_epochs = 4
  learning_rate = 1e-3
  n_prompt_tokens = 10
  random_range  = 0.5
  batch_size = 64
  max_grad_norm = 0.1
args = Config()


In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True, padding_value=0)
    labels = torch.stack([item['labels'] for item in batch])

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [ ]:
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType


from datasets import load_dataset
raw_datasets = load_dataset("glue", "sst2")


from transformers import AutoTokenizer

checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, num_labels=2)


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["sentence"], max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")



train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)


In [ ]:

import numpy as np

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()

    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:
        #batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss, logits = outputs[:2]

        preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        labels =batch['labels'].detach().cpu().numpy()

        loss_arr.append(loss.item())
        accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

In [ ]:
LOGGING_INTERVAL = 100

In [ ]:
def train_prefix(model,optimiser, train_dataloader):
  for epoch in range(1, args.num_train_epochs+1):
    losses = []

    for  step, batch in enumerate(train_dataloader):
            optimiser.zero_grad()
            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # output = loss, logits, hidden_states, attentions
            loss = outputs[0]

            loss.backward()
            losses.append(loss.item())
            optimiser.step()

            if step > 0 and step % LOGGING_INTERVAL == 0:
                train_loss = np.mean(losses)
                eps = privacy_engine.get_epsilon(DELTA)
                eval_loss, eval_accuracy = evaluate(model)

                print(
                  f"Epoch: {epoch} | "
                  f"Step: {step} | "
                  f"Train loss: {train_loss:.3f} | "
                  f"Eval loss: {eval_loss:.3f} | "
                  f"Eval accuracy: {eval_accuracy:.3f} | "
                  f"ɛ: {eps:.2f}"
                )

In [ ]:
from transformers import  AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=10)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
from torch.optim import AdamW

optimiser = AdamW(model.parameters(), lr=args.learning_rate)

In [ ]:
DELTA = 1/len(train_dataloader)

In [ ]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


In [ ]:
train_prefix(model, optimiser, train_dataloader)

QNLI

In [ ]:
raw_datasets = load_dataset("glue", "qnli")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])

raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question"],example["sentence"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'sentence', 'question'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=10)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = get_peft_model(model, peft_config)

In [ ]:
optimiser = AdamW(model.parameters(), lr=args.learning_rate)

In [ ]:
DELTA = 1/len(train_dataloader)

In [ ]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


In [ ]:
train_prefix(model, optimiser, train_dataloader)

QQP

In [ ]:
raw_datasets = load_dataset("glue", "qqp")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])

raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question1"],example["question2"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'question1', 'question2'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
del model
del optimiser

In [ ]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=10)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = get_peft_model(model, peft_config)

In [ ]:
optimiser = AdamW(model.parameters(), lr=args.learning_rate)

In [ ]:
DELTA = 1/len(train_dataloader)

In [ ]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)


In [ ]:
train_prefix(model, optimiser, train_dataloader)

MNLI

In [ ]:
raw_datasets = load_dataset("glue", "mnli")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation_matched'] = raw_datasets['validation_matched'].select([i for i in range(5000)])



tokenizer = AutoTokenizer.from_pretrained(checkpoint, num_labels =3)


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["premise"],example["hypothesis"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'premise', 'hypothesis'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation_matched"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
del model
del optimiser

In [ ]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, num_virtual_tokens=10)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
model = get_peft_model(model, peft_config)

In [ ]:
optimiser = AdamW(model.parameters(), lr=args.learning_rate)

In [ ]:
DELTA = 1/len(train_dataloader)

In [ ]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)

In [ ]:
train_prefix(model, optimiser, train_dataloader)